# Working with Data via MXNet NDArray

MXNet's NDArray is a tensor data structure similar to numpy's multi-dimensional array. 
It confers a few key advantages. First, it supports asynchronous computation on CPU and GPU, and distributed cloud architectures. Second, it provides support for automatic differentiation. These properties make it especially suited as a library for both deep learning research and production.


## Getting Started

First, let's import mxnet and (for convenience) mxnet.ndarray, the only dependencies we'll need in this tutorial.

In [ ]:
import mxnet as mx
import mxnet.ndarray as nd

First, we'll create a 6x4 matrix, without initializing values

In [ ]:
x = nd.empty(shape=(6,4))
print(x)

We can also create a 6x5 matrix, initialized with values drawn from a standard normal distribution.

In [ ]:
x = nd.random_normal(shape=(6,4))
print(x)

As in NumPy, we can get its shape

In [4]:
print(x.shape)

(6, 4)


We can also query it's size, which is equal to the product of the shape.

In [5]:
print(x.size)

24


## Operations

There are multiple syntaxes for operations. Let’s see addition as an example

In [10]:
y = nd.random_normal(shape=(6,4))
c = x + y
print(c)

[[ 2.99384356  3.28537607 -0.66300189  0.89417839]
 [ 0.35340142  0.07233012 -2.35445523  0.40822887]
 [-1.76046467  1.10349369  2.41667604 -0.49296868]
 [-0.77388644 -3.24447298 -2.15431499 -1.52447724]
 [ 0.89149493 -0.46309918  3.2145586  -3.18396688]
 [ 2.77407885 -3.45184565  1.35531378 -0.73216116]]
<NDArray 6x4 @cpu(0)>


In the previous example, MXNet allocates new memory for the sum ``x+y`` and assigns a reference to the variable ``c``. To make better use of memory, we often prefer to perform operations in place, reusing already allocated memory. 

In MXNet, we can perform inplace operations by assigning the results of operations using slice notation (``result[:] = ...``).

In [12]:
result = nd.zeros(shape=(6,4))
result[:] = x+y
print(result)

[[ 2.99384356  3.28537607 -0.66300189  0.89417839]
 [ 0.35340142  0.07233012 -2.35445523  0.40822887]
 [-1.76046467  1.10349369  2.41667604 -0.49296868]
 [-0.77388644 -3.24447298 -2.15431499 -1.52447724]
 [ 0.89149493 -0.46309918  3.2145586  -3.18396688]
 [ 2.77407885 -3.45184565  1.35531378 -0.73216116]]
<NDArray 6x4 @cpu(0)>


## Slicing

MXNet NDArrays currently support slicing along the first (0th) axis only. To use arbitrary slicing patterns, you can convert NDArrays to numpy arrays and back.

In [13]:
result[2:4]

[[-1.76046467  1.10349369  2.41667604 -0.49296868]
 [-0.77388644 -3.24447298 -2.15431499 -1.52447724]]
<NDArray 2x4 @cpu(0)>

## Converting from MXNet NDArray to NumPy Array

Converting MXNet NDArrays to and from NumPy is easy. Note that the converted arrays do not share memory.

In [14]:
a = nd.ones(shape=(5))
print(a)

[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


In [15]:
b = a.asnumpy()
print(b)

[ 1.  1.  1.  1.  1.]


In [16]:
b[0] = 2
print(b)
print(a)

[ 2.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


## Converting from NumPy Array to MXNet NDArray

Constructing an MXNet NDarray from a NumPy Array is straightforward.

In [17]:
c = nd.array(b)
print(c)

[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


## Managing Context

In MXNet, every array has a context. A context could be the CPU, or one of many GPUs. By assigning arrays to contexts intelligently, we can minimize the time spent transferring data between devices. For example, when training neural networks on a server with a GPU, we typically prefer for the model's parameters to live on the GPU. 


In [19]:
d = nd.array(b, mx.cpu())

Given an NDArray on a given context, we can copy it to another context by using the ``copyto()`` method.

In [20]:
e = d.copyto(mx.cpu(1))
print(e)

[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(1)>


Often we only want to make a copy if the variable isn't already on the desired context. In these cases, we can call ``as_in_context()`` which only makes a copy when the source and target contexts are different.

In [22]:
f = d.as_in_context(mx.cpu(0))
print(f)

[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>
